# NIPS 2017 Figure 4

In [ ]:
%run -i NIPS2017.ipynb

# FIGURE and GRID
FIG_HEIGHT_MM = 65
FIG_WIDTH_MM = FIG_WIDTH_MM  # set in NIPS2017 notebook to a default value for all figures
FIG_N_ROWS = 2
ROW_1_NCOLS = 3
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_3_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_2_NCOLS = 3
ROW_2_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_3_MM = FIG_WIDTH_MM / ROW_2_NCOLS

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig4_a.svg'
PANEL_B = PATH_DROPBOX_FIGS + 'fig4_b.svg'
PANEL_C = PATH_DROPBOX_FIGS + 'fig4_c.svg'
PANEL_D = PATH_DROPBOX_FIGS + 'fig4_d.svg'
PANEL_E = PATH_DROPBOX_FIGS + 'fig4_e.svg'
PANEL_F = PATH_DROPBOX_FIGS + 'fig4_f.svg'

## Row 1

Generate and load data

In [ ]:
from lfmods.hh import HHSimulator

sim = HHSimulator(seed=100,pilot_samples=0.,cached_sims=False, cached_pilot=False)

# ground-truth parameters
true_params = sim.true_params
n_params = sim.n_params

init = sim.init
t = sim.t
dt = sim.dt
duration = np.max(t)
I = sim.I_obs

# prior over parameters
sim_prior = sim.sim_prior
sim_prior()
prior_min = sim.prior_min
prior_max = sim.prior_max

# if parameters in transformed space, inverse transform function
param_invtransform = sim.param_invtransform

bm = sim.bm

# simulate
fact = 0.2
new_prior_min = fact*prior_min
new_prior_max = fact*prior_max
np.random.seed(123)
z = np.random.rand(n_params)
params = param_invtransform( np.multiply(new_prior_max - new_prior_min,z) + new_prior_min )
hh = bm.HH(init, params.reshape(1,-1),seed=230)
states_bd = hh.sim_time(dt, t, I)[:,0]

### Panel A

In [ ]:
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, states_bd, color=COL['GT'], lw=3)
    plt.ylabel('voltage (mV)')

    ax.set_xlim([0, duration])
    ax.set_xticks([])

    ax = plt.subplot(gs[1])
    plt.plot(t,I,'r', lw=3)
    plt.xlabel('')
    plt.ylabel('input (mA)')

    ax.set_xlim([0, duration])
    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%d ms'))

    plt.savefig(PANEL_A, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg

svg(PANEL_A)

panel = st.fromfile(PANEL_A)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel B

In [ ]:
try:
    svg(PANEL_B)
except:
    pass

### Panel C

In [ ]:
try:
    svg(PANEL_C)
except:
    pass

## Row 2

In [ ]:
from lfmods.hh import HHSimulator

sim = HHSimulator(seed=100,pilot_samples=0.,cached_sims=False, cached_pilot=False)

# ground-truth parameters
true_params = sim.true_params
n_params = sim.n_params

init = sim.init
t = sim.t
dt = sim.dt
duration = np.max(t)
I = sim.I_obs

# prior over parameters
sim_prior = sim.sim_prior
sim_prior()
prior_min = sim.prior_min
prior_max = sim.prior_max

# if parameters in transformed space, inverse transform function
param_invtransform = sim.param_invtransform

bm = sim.bm

# simulate
fact = .9
new_prior_min = fact*prior_min
new_prior_max = fact*prior_max
np.random.seed(123)
z = np.random.rand(n_params)
params = param_invtransform( np.multiply(new_prior_max - new_prior_min,z) + new_prior_min )
hh = bm.HH(init, params.reshape(1,-1),seed=230)
states_mf = hh.sim_time(dt, t, I)[:,0]

### Panel D

In [ ]:
fig_inches = (SCALE_IN*mm2inches(ROW_2_WIDTH_COL_1_MM), SCALE_IN*mm2inches(ROW_2_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, states_mf, color=COL['GT'], lw=3)
    plt.ylabel('voltage (mV)')

    ax.set_xlim([0, duration])
    ax.set_xticks([])
    ax.set_yticks([-80, -20, 40])

    ax = plt.subplot(gs[1])
    plt.plot(t,I,'r', lw=3)
    plt.xlabel('')
    plt.ylabel('input (mA)')

    ax.set_xlim([0, duration])
    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%d ms'))

    plt.savefig(PANEL_D, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg

svg(PANEL_D)

panel = st.fromfile(PANEL_D)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel E

### Panel F

## Compose figure

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)

yoffset = -1.2
xoffset = -1.
fig = add_svg(fig, PANEL_A, 0 + xoffset, 0 + yoffset)
#fig = add_svg(fig, PANEL_B, ROW_1_WIDTH_COL_1_MM + xoffset, 0 + yoffset)
#fig = add_svg(fig, PANEL_C, ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + xoffset, 0 + yoffset)
fig = add_svg(fig, PANEL_D, 0 + xoffset, ROW_2_HEIGHT_MM + yoffset)
#fig = add_svg(fig, PANEL_E, ROW_2_WIDTH_COL_1_MM + xoffset, ROW_2_HEIGHT_MM + yoffset)
#fig = add_svg(fig, PANEL_F, ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM + xoffset, ROW_2_HEIGHT_MM + yoffset)

yoffset = 2.3
fig = add_label(fig, 'A', 0, 0 + yoffset)
fig = add_label(fig, 'B illustration solution', ROW_1_WIDTH_COL_1_MM, 0 + yoffset)
fig = add_label(fig, 'C results', ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM, 0 + yoffset)
yoffset = 1.5
fig = add_label(fig, 'D', 0, ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'E illustration solution', ROW_2_WIDTH_COL_1_MM, ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'F results', ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM, ROW_2_HEIGHT_MM + yoffset)

PATH_SVG = PATH_DROPBOX_FIGS + 'fig4.svg'
fig.save(PATH_SVG)

svg(PATH_SVG)

!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig4.pdf $PATH_SVG